In [ ]:
from modules.resnet import ResNet
import numpy as np
import tensorflow as tf
from modules import feature_extractor as fe
from modules import modelCreator
from modules.dataset_selecter import datasetSelector
from sklearn.model_selection import StratifiedKFold

import os
import random
from tensorflow.python.eager import context
from tensorflow.python.framework import ops

seed_value = 56

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
np.random.RandomState(seed_value)
np.random.seed(seed_value)
context.set_global_seed(seed_value)
ops.get_default_graph().seed = seed_value

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
np.random.seed(seed_value)


def run_experiments(seed_value=56, 
                    takeName=False, use_cache=True, saveMethod='pickle',
                    skipDebugSaves=True, saveWeights=False,
                    nrFolds=5, batchSize=100, numEpochs=500, patience=70,
                    numbers=[1], 
                    method='TSFRESH_SEL_SYM', symbolCount=0, strategy=None, ncoefI=None, coef_div=None, 
                    architecture='transformer', numOfAttentionLayers=None, header=None, dmodel=None, dff=None, dropOutRate=None, lr=None, warmup=None, 
                    num_resblocks=None, num_channels=None, use_1x1conv=None):
    
    for number in numbers:

        kf = StratifiedKFold(nrFolds, shuffle=True, random_state=seed_value)

        X_train, X_test, y_train, y_test, y_trainy, y_testy, seqSize, dataName, num_of_classes, number = datasetSelector('ucr', seed_value, number, takeName, use_cache)


        saved = fe.save_foldwise_features(number, dataName, method, X_train, X_test, y_train, y_test, y_trainy, y_testy, kf)
        fold = 0

        abstractionString = method
        fullResults = dict()

        accuracies = []

        for train, test in kf.split(X_train, y_trainy):
            fold += 1
            print("Starting fold: ", fold)

            x_train1, x_val, x_test, y_train1, y_val, y_test, y_trainy1, y_valy, y_testy = fe.load_foldwise_features(number, dataName, method, fold, saveMethod)
                
            if np.any(np.isnan(x_train1.flatten())) or np.any(np.isnan(x_val.flatten())) or np.any(np.isnan(x_test.flatten())) or np.any(np.isnan(y_train1.flatten())) or np.any(np.isnan(y_val.flatten())) or np.any(np.isnan(y_test.flatten())):
                continue

            if x_train1 is None:
                print("No relevant features extracted, skipping..")
                continue 
            
            x_train1, x_val, x_test, y_train1, y_val, y_test, X_train_ori, X_val_ori, X_test_ori, y_trainy1, y_testy = modelCreator.preprocessData(x_train1, x_val, x_test, y_train1, y_val, y_test, 
                                                                                                                                                y_trainy1, y_testy, fold, symbolsCount=symbolCount, dataName=dataName, 
                                                                                                                                                seqSize=seqSize,
                                                                                                                                                strategy=strategy,
                                                                                                                                                method=method,
                                                                                                                                                ncoef=ncoefI, coef_div=coef_div)

            out = modelCreator.doAbstractedTraining(x_train1, x_val, x_test, y_train1, y_val, y_testy, 
                                                    BATCH=batchSize, seed_value=seed_value, num_epochs=numEpochs, patience = patience, 
                                                    num_of_classes=num_of_classes, number=number, dataName=dataName, fold=fold,
                                                    architecture=architecture, symbolCount=symbolCount, 
                                                    numOfAttentionLayers=numOfAttentionLayers,header=header, dmodel=dmodel, dff=dff, 
                                                    num_resblocks=num_resblocks, num_channels=num_channels, use_1x1conv=use_1x1conv,
                                                    abstractionType=abstractionString, rate=dropOutRate, lr=lr, warmup=warmup, 
                                                    skipDebugSaves=skipDebugSaves, saveWeights=saveWeights)
            
            saveName = modelCreator.getWeightName(number, dataName, 0,
                                                    learning = False, results = True,
                                                    architecture=architecture, abstractionType=method, symbols=symbolCount,
                                                    layers=numOfAttentionLayers, header=header, dmodel=dmodel, dff=dff,
                                                    num_resblocks=num_resblocks, num_channels=num_channels, use_1x1conv=use_1x1conv,
                                                    num_coef=ncoefI, dropout=dropOutRate, strategy=strategy)

            
            print("finished fold: " + str(fold))
            print("architecture")
            print("Method: ", method)
            print("Dataset: ", number)
            print("Fold: ", fold)
            print("Accuracy: ", out[1][0])
            accuracies.append(out[1][0])
            print(saveName)

        print("Final results")
        print("Method: ", method)
        print("Dataset: ", number)
        print("Average accuracy: ", np.mean(accuracies))

# dataset parameters
takeName = False
use_cache = True
saveMethod = 'pickle'

# additional saves
skipDebugSaves = True
saveWeights = False

# training parameters
nrFolds = 5
batchSize = 100
numEpochs = 500
patience = 70

# dataset numbers
numbers = [1]

# method
method = 'TSFRESH_SEL_SYM'
symbolCount = 5
strategy = 'uniform'
ncoefI = 0 # needed for SFA
coef_div = 0 # needed for SFA

# architecture 'transformer' or 'resnet'

### transformer model parameters
architecture = 'transformer'
numOfAttentionLayers = 3
header = 4
dmodel = 32
dff = 16
dropOutRate = 0.0
lr = 'custom'
warmup = 10000

## resnet model parameters
# architecture = 'resnet'
# num_resblocks = 3
# num_channels = 16
# use_1x1conv = True
# lr = 0.0001

run_experiments(seed_value=seed_value, 
                takeName=takeName, use_cache=use_cache, saveMethod=saveMethod,
                skipDebugSaves=skipDebugSaves, saveWeights=saveWeights,
                nrFolds=nrFolds, batchSize=batchSize, numEpochs=numEpochs, patience=patience,
                numbers=numbers, 
                method=method, symbolCount=symbolCount, strategy=strategy, ncoefI=ncoefI, coef_div=coef_div, 
                architecture=architecture, 
                numOfAttentionLayers=numOfAttentionLayers, header=header, dmodel=dmodel, dff=dff, dropOutRate=dropOutRate, lr=lr, warmup=warmup)
                # num_resblocks=num_resblocks, num_channels=num_channels, use_1x1conv=use_1x1conv, lr=lr)

---------------------------------- SAVING FEATURES ----------------------------------
---------------------------------- FOLD 1 ----------------------------------
File already exists
---------------------------------- FOLD 2 ----------------------------------
File already exists
---------------------------------- FOLD 3 ----------------------------------
File already exists
---------------------------------- FOLD 4 ----------------------------------
File already exists
---------------------------------- FOLD 5 ----------------------------------
File already exists
Starting fold:  1


2025-02-28 14:53:44.601986: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
I0000 00:00:1740750826.978841  805046 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step

2025-02-28 14:54:07.276208: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
test fold score with input TSFRESH_SEL_SYM-(accuracy): 0.5933503836317136
finished fold: 1
architecture
Method:  TSFRESH_SEL_SYM
Dataset:  1
Fold:  1
Accuracy:  0.5933503836317136
./results/results-1_Adiac -a resnet -m TSFRESH_SEL_SYM -s 5 -rb 3 -ch 16 -cv 1 -st u -cf 256.h5
Starting fold:  2
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
test fold score with input TSFRESH_SEL_SYM-(accuracy): 0.5882352941176471
finished fold: 3
architecture
Method:  TSFRESH_SEL_SYM
Dataset:  1
Fold:  3
Accuracy:  0.5882352941176471
./results/results-1_Adiac -a resnet -m TSFRESH_SEL_SYM -s 5 -rb 3 -ch 16 -cv 1 -st u -cf 256.h5
Starting fold:  4
 1/10 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step

2025-02-28 14:55:09.908641: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
test fold score with input TSFRESH_SEL_SYM-(accuracy): 0.680306905370844
finished fold: 4
architecture
Method:  TSFRESH_SEL_SYM
Dataset:  1
Fold:  4
Accuracy:  0.680306905370844
./results/results-1_Adiac -a resnet -m TSFRESH_SEL_SYM -s 5 -rb 3 -ch 16 -cv 1 -st u -cf 256.h5
Starting fold:  5
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step

2025-02-28 14:55:31.167696: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
test fold score with input TSFRESH_SEL_SYM-(accuracy): 0.6342710997442456
finished fold: 5
architecture
Method:  TSFRESH_SEL_SYM
Dataset:  1
Fold:  5
Accuracy:  0.6342710997442456
./results/results-1_Adiac -a resnet -m TSFRESH_SEL_SYM -s 5 -rb 3 -ch 16 -cv 1 -st u -cf 256.h5
Final results
Method:  TSFRESH_SEL_SYM
Dataset:  1
Average accuracy:  0.6265984654731458
